
##### refunder stream

this notebook starts a stream to score completed orders for potential refunds

In [0]:
%pip install --upgrade databricks-sdk

In [0]:
#### TODO: for some reason the experiments tracing page is not registering the calls made through this, investigate that
###         update: i believe this happens because the experiment created by the task of this job (see ./jobs/refund_recommender_stream) comes from a "notebook" that is ephemeral (exists only for the scope of the job) and so inference calls outside the runtime of the job go nowhere

In [ ]:
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.jobs as j
import os

w = WorkspaceClient()

CATALOG = dbutils.widgets.get("CATALOG")

notebook_abs_path = os.path.abspath("../jobs/refund_recommender_stream")
notebook_dbx_path = notebook_abs_path.replace(
    os.environ.get("DATABRICKS_WORKSPACE_ROOT", "/Workspace"),
    "/Workspace"
)

job_name = f"Refund Recommender Stream ({CATALOG})"

task_def = [
    j.Task(
        task_key="refund_recommender_stream",
        notebook_task=j.NotebookTask(
            notebook_path=notebook_dbx_path,
            base_parameters={
                "CATALOG": CATALOG,
                "REFUND_AGENT_ENDPOINT_NAME": dbutils.widgets.get("REFUND_AGENT_ENDPOINT_NAME"),
            },
        )
    )
]
schedule_def = j.CronSchedule(
    quartz_cron_expression="0 0/10 * * * ?",
    timezone_id="UTC",
    pause_status=j.PauseStatus.UNPAUSED,
)

existing = [jb for jb in w.jobs.list(name=job_name) if jb.settings.name == job_name]
if existing:
    job_id = existing[0].job_id
    w.jobs.reset(job_id=job_id, new_settings=j.JobSettings(
        name=job_name, tasks=task_def, schedule=schedule_def,
    ))
    print(f"♻️ Updated existing job_id={job_id}")
else:
    job = w.jobs.create(name=job_name, tasks=task_def, schedule=schedule_def)
    job_id = job.job_id
    import sys
    sys.path.append('../utils')
    from uc_state import add
    add(CATALOG, "jobs", job)
    print(f"✅ Created job_id={job_id}")

w.jobs.run_now(job_id=job_id)
print(f"🚀 Started run of {job_name}")
